# MCML Lab 1

----------------

## Reference
- Software: http://omlc.org/software/mc/
- MCML, "terse but complete description of the Monte Carlo simulation and convolution" http://omlc.org/~prahl/pubs/pdf/prahl89.pdf
- MCML manual, http://omlc.org/software/mc/mcml/MCman.pdf

----------------

## MCML 的數學原理
### Introduction


----------------


## 檔案結構

### MCML 的 檔案結構分別爲 
- **mcml.h:**
  - 主要是 header 檔，定義了一些 struct 如一些常數 pi， photon 的結構，layer 的結構，
- **mcmlmain.c: **
  - 主程式 main()，主要作爲 call function 的界面，以及時間的控制
- **mcmlgo.c: **
  - monte carlo simulation 的 主要程式碼
- **mcmlnr.c: **
  - 主要有 allocate/free vector/matrix 以及 report error message 的 function
- **mcmlio.c: **
  - 主要的 io 界面


----------------


## Detailed

### Main function

- while num of photons != 0
  - LaunchPhoton
  - while photon not dead
    - HopDropSpin (simulate the photon propagate behaviour)
  - num of photons -1
  

----------------


### MCMLGO.C

- **Numerical Issue:**
  - define COSZERO (1.0-1.0E-12)
  - define COS90D  1.0E-6
- ** What does ran3(int *idum) do? **
  - A random number generator from Numerical Recipes in C
  - RandomNum:  Generate a random number between 0 and 1.
  - The seed is limited to 1<<15. 
- **Rspecular:**  Compute the specular reflection
  - If the first layer is a turbid medium, use the Fresnel reflection from the boundary of the first layer as the specular reflectance.  $$R_{sp} = \frac{(n_1-n_2)^2}{(n_1+n_2)^2}$$
  - If the first layer is glass, multiple reflections in the first layer is considered to get the specular reflectance. $$R_{sp} = n_1 + \frac{(1-n_1)^2 n_2}{(1-n_1n_2)}$$
- **LaunchPhoton**: Initialize a photon packet
  - photon packet decreased by Rspecular:  $$W = 1- R_{sp}$$
- **SpinTheta: (Scattering angle)** 
  - sample a new theta angle for photon propagation according to the anisotropy (Henyey-Greenstein)
  - If anisotropy g is 0,
    - $cos(\theta) = 2*RandomNum() -1;$
  - else, 
    - $cos(\theta) = 1/(2g) \{1+g^2-\frac{1-g^2}{1-g+2g\xi}\}$ , where the $\xi = RandomNum()$
- **Spin: (new direction)** 
  - Choose a new direction for photon propagation by sampling the polar deflection angle theta and the azimuthal angle psi.
- **Hop:** 
  - Move the photon away in the current layer of medium
- **StepSizeInGlass:**
  - If uz != 0, return the photon step size in glass, where the step size is the distance between the current position and the boundary in the photon direction
  - Make sure uz !=0 before calling this function
- **StepSizeInTissue:** 
  - Pick a step size for a photon packet when it is in tissue. If the member sleft is zero, make a new step size with: $$-log(rnd)/(mua+mus).$$
- **HitBoundary: **
  - Check if the step will hit the boundary. 
  - If the projected step hits the boundary, the members s and sleft of Photon_Ptr are updated.  p/s: s is step size
- **Drop: **
  - Drop photon weight inside the tissue (not glass). 
  - The photon is assumed not dead. 
  - The weight drop is $dw = w*mua/(mua+mus).$
  - The dropped weight is assigned to the absorption array elements
  - Numerical issue of index to z and r
- **Roulette: **
  - The photon weight is small, and the photon packet tries to survive a roulette.
  - $RandomNum() < CHANCE$ : determined by chance (default 0.1)
- **RFresnel:** 
  - Compute the Fresnel reflectance
  - Make sure that the cosine of the incident angle is positive, and the case when the angle is greater is positive, and the case when the angle is greater
  - **Avoid trigonometric function operations as much as possible**, because they are computation-intensive
  - Input given cos angle etc..
  - The equation of 
  $$R(\theta_i)=\frac{1}{2}[\frac{sin^2(\theta_i-\theta_t)}{sin^2(\theta_i+\theta_t)}+\frac{tan^2(\theta_i-\theta_t)}{tan^2(\theta_i+\theta_t)}] = \frac{1}{2} sin^2(\theta_i-\theta_t)[\frac{cos^2(\theta_i-\theta_t)+cos^2(\theta_i+\theta_t)}{sin^2(\theta_i+\theta_t)}]$$
  - By using the rule: 
    - $sin(x \pm y) = sin x cos y \pm cos x sin y$
    - $cos(x \mp y) = cos x cosy \pm sin x sin y$
- **RecordR:**
  - Record the photon weight exiting the first layer($uz<0$),no matter whether the layer is glass or not, to the reflection array
- **RecordT:**
  - Record the photon weight exiting the first layer($uz<0$),no matter whether the layer is glass or not, to the transmittance array
- **CrossUpOrNot:**
  - Decide whether the photon will be transmitted or reflected on the upper boundary (uz<0) of the current layer
  -	If "layer" is the first layer, the photon packet will be partially transmitted and partially reflected if PARTIALREFLECTION is set to 1,or the photon packet will be either transmitted or reflected determined statistically if PARTIALREFLECTION is set to 0.
  - Record the transmitted photon weight as reflection
  - If the "layer" is not the first layer and the photon packet is transmitted, move the photon to "layer-1".
- **CrossDnOrNot:**
  - Decide whether the photon will be transmitted or reflected on the bottom boundary (uz>0) of the current layer
  - If the photon is transmitted, move the photon to "layer+1". If "layer" is the last layer, record the transmitted weight as transmittance. See comments for CrossUpOrNot.
- **HopInGlass:**
  - Move the photon packet in glass layer. 
  - Horizontal photons are killed because they will never interact with tissue again.
- **HopDropSpinInTissue:**
  - Set a step size, move the photon, drop some weight, choose a new photon direction for propagation.
  - When a step size is long enough for the photon to hit an interface, this step is divided into two steps. 
      - First, move the photon to the boundary free of absorption or scattering, then decide whether the photon is reflected or transmitted.
      - Then move the photon in the current or transmission medium with the unfinished stepsize to interaction site.  If the unfinished stepsize is still too long, repeat the above process.
- ** HopDropSpin **:
  - combine HopInGlass and HopDropSpinInTissue if the photon in different medium.




----------------


## Potential Speed up

* 因爲個別的 photon 的行爲是 independent 的，因此可以同時做 N 個 photon 的 monte carlo 的模擬，$N>1$
* 除此之外其他的地方看起來都是sequential 的，無法對其他部分做平行化